In [1]:
import netCDF4 as nc
import numpy as np
import pandas as pd 
import os

import dash
import dash_core_components as dcc
import dash_html_components as html

path = 'E:/Users/LuoZN/Desktop/Data_vis'
equalarea = 'Raw_TAVG_EqualArea.nc'
grid = 'Raw_TAVG_LatLong1.nc'

In [2]:
file_path = os.path.join(path,equalarea)
file_obj = nc.Dataset(file_path)
file_obj

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: Berkeley Earth Internal Convention (based on CF-1.5)
    title: Native Format Berkeley Earth Surface Temperature Anomaly Field
    history: 23-Jan-2020 11:11:30
    institution: Berkeley Earth Surface Temperature Project
    source_file: NoScalpelNoWeighting_TAVG.50985s.20200123T110933.mat
    source_history: 23-Jan-2020 07:33:02
    source_data_version: ca6f26341938dae0ea7dd619bce6f15e
    comment: This file contains Berkeley Earth surface temperature anomaly field in our native equal-area format.
    dimensions(sizes): map_points(5498), time(3240), month_number(12)
    variables(dimensions): float32 longitude(map_points), float32 latitude(map_points), float64 time(time), float64 land_mask(map_points), float32 temperature(time,map_points), float32 climatology(month_number,map_points)
    groups: 

In [3]:
file_obj.variables.keys()

odict_keys(['longitude', 'latitude', 'time', 'land_mask', 'temperature', 'climatology'])

In [4]:
longitude = file_obj.variables['longitude']
latitude = file_obj.variables['latitude']
time = file_obj.variables['time']
land_mask = file_obj.variables['land_mask']
temperature = file_obj.variables['temperature']
climatology = file_obj.variables['climatology']

In [5]:
def get_xyz(latitude,longitude,R):
    lat = latitude/np.pi
    long = longitude/np.pi
    x = R * np.math.cos(lat) * np.math.cos(long)
    y = R * np.math.cos(lat) * np.math.sin(long)
    z = R * np.math.sin(lat)
    return x,y,z
get_xyz(0,120,40)

(35.14016275371421, 19.109394591207668, 0.0)

In [33]:
tp = np.array(temperature)
tp = pd.DataFrame(tp)
tp['time_id']=tp.index
tp = tp.melt(id_vars='time_id')
tp = tp.rename(columns={'variable':'pt_id','value':'temperature'})

lat,lon = np.array(latitude),np.array(longitude)
lat,lon = pd.DataFrame(lat),pd.DataFrame(lon)
lat['pt_id'] = lat.index
lat = lat.rename(columns={0:'latitude'})
lon['pt_id'] = lon.index
lon = lon.rename(columns={0:'longitude'})

t = np.array(time)
t = pd.DataFrame(t)
t['time_id'] = t.index
t = t.rename(columns={0:'time'})

D:\Users\LuoZN\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning:

invalid value encountered in less

D:\Users\LuoZN\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning:

invalid value encountered in greater



In [34]:
df = tp.merge(t,how='left',on='time_id').merge(lat,how='left',on='pt_id').merge(lon,how='left',on='pt_id')
df 

,time_id,pt_id,temperature,time,latitude,longitude
0,0,0,NaN,1750.041667,82.788071,-83.571426
1,1,0,NaN,1750.125000,82.788071,-83.571426
2,2,0,NaN,1750.208333,82.788071,-83.571426
3,3,0,NaN,1750.291667,82.788071,-83.571426
4,4,0,NaN,1750.375000,82.788071,-83.571426
...,...,...,...,...,...,...
17813515,3235,5497,-0.661482,2019.625000,-89.215027,120.000000
17813516,3236,5497,-0.035485,2019.708333,-89.215027,120.000000
17813517,3237,5497,-0.595212,2019.791667,-89.215027,120.000000
17813518,3238,5497,0.651016,2019.875000,-89.215027,120.000000


In [118]:
df = []
for i in range(5498):
    lat = latitude[i]
    long = longitude[i]
    tp = float(temperature[-1,i].data)
    df.append([lat,long,tp])
df = pd.DataFrame(df,columns=['latitude','longitude','temperature'])
df

,latitude,longitude,temperature
0,82.78807,-83.57143,4.824538
1,82.78807,-70.71429,4.452980
2,82.78807,-57.857143,4.093904
3,82.78807,-45.0,3.769627
4,82.78807,-32.142857,3.196401
...,...,...,...
5493,-87.644936,120.0,0.028248
5494,-87.644936,160.0,0.467041
5495,-89.21503,-120.0,0.795195
5496,-89.21503,0.0,0.392949


In [82]:
df['temperature'][0]

array(4.824538, dtype=float32)

In [ ]:
import plotly.express as px
#df = px.data.gapminder()
fig = px.scatter_geo(df, lat = 'latitude', lon = 'longitude', color="temperature",
               projection="orthographic",animation_frame ='time')
fig.show()

In [100]:
import plotly.express as px
#df = px.data.election()
fig = px.scatter_3d(df, x="x", y="y", z="z",color='temperature')
fig.show()

In [126]:
from geopy.geocoders import Nominatim

geolocator = Nominatim()
location = geolocator.reverse("39.9073426, 60.391264649167")   # 传入纬度、经度字符串
print(location.address)
print((location.latitude,location.longitude))
print(location.altitude)
print(location.raw)

D:\Users\LuoZN\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning:

Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.



Daşoguz welaýaty, Türkmenistan
(41.1582796, 58.71143523158547)
0.0
{'place_id': 235089873, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 223028, 'lat': '41.1582796', 'lon': '58.71143523158547', 'display_name': 'Daşoguz welaýaty, Türkmenistan', 'address': {'state': 'Daşoguz welaýaty', 'country': 'Türkmenistan', 'country_code': 'tm'}, 'boundingbox': ['39.4672693', '42.797427', '56.4293754', '60.9448991']}


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Download data set from plotly repo
pts = np.loadtxt(np.DataSource().open('https://raw.githubusercontent.com/plotly/datasets/master/mesh_dataset.txt'))
x, y, z = pts.T

fig = go.Figure(data=[go.Mesh3d(x=x, y=y, z=z, color='lightpink', opacity=0.50)])
fig.show()